In [1]:
import pandas as pd
import numpy as np

## Descripción del problema a resolver

**La base de datos** que se utilizará para probar el paquete implementado por el equipo corresponde a una representación simplificada de la Red Eléctrica Mexicana, que se utiliza para realizar la planeación del sistema nacional de generación, transmisión y distribución de energía eléctrica.

La información a la que se tuvo acceso proviene del Centro Nacional de Control de Energía ([CENACE](https://www.gob.mx/cenace)) y su publicación se realiza de forma anual en los Programas de Ampliación y Modernización de la Red Nacional de Transmisión y Redes Generales de Distribución ([ver documento PAMRNT](https://www.cenace.gob.mx/Docs/10_PLANEACION/ProgramasAyM/Programa%20de%20Ampliaci%C3%B3n%20y%20Modernizaci%C3%B3n%20de%20la%20RNT%20y%20RGD%202021%20-%202035.pdf)).

En la Figura 4.3.2 de ese documento, se muestra la topología que tiene la red que representa las regiones o zonas más representativas (en cuanto a demanda y generación de energía eléctrica o bien por cuestiones de ubicación geográfica), así como su conectividad. Adicionalmente, cada uno de los arcos (ramas) tiene una capacidad definida de transmisión de energía, comunmente llamada _límite de transmisión entre regiones_.

<p align = "center">
    <img src="images/red_nacional.png" width="1329" height="911" />

        fuente: Elaborado por CENACE

El grafo que observamos es de tipo "no-dirigido", porque en una red eléctrica el sentido del flujo de potencia (energía) puede darse en cualquier sentido y está determinado por la solución que se obtenga del problema de [Flujos de Potencia](https://www.intechopen.com/chapters/65445). Sin embargo, para el ejercicio que realizaremos en esta práctica, partiremos de una suposición de sentido en los flujos de potencia basada en las condiciones que predominan en la red eléctrica y que se reportan en el PAMRNT (en la sección: _Condiciones operativas en las transferencias de potencia en los principales enlaces del Sistema Eléctrico Nacional en la demanda máxima de verano de 2020_).

**El Planteamiento** del problema que se pretende resolver es el siguiente: "Dada la red eléctrica de la Figura 4.3.2 encontrar flujo máximo que se puede transmitir en la red".

**¿Para qué puede servir encontrar el flujo máximo en la red eléctrica?**

- Para determinar los posibles cuellos de botella (restricciones) que se pueden presentar al tratar de enviar energía desde un punto de la red a otro.
- Encontrar posibles puntos de inyección donde resulte más conveniente instalar generación (que se obtengan mayores flujos máximos por la red)
- Descubrir cuales corredores de trasnmisión (rutas) se ven más utilizadas cuando la inyección de energía se presenta en algún punto de la red.


## Lectura y limpieza de la Base de Datos

La base de datos tiene el siguiente contenido y forma:

In [113]:
red = pd.read_csv('BD/red.csv')
pd.options.display.max_rows = 70
red

,Num_env,Nom_env,Num_rec,Nom_rec,Enlace,Periodo,Cap,Real,Img
0,24,1ROMAYO,31,AGUASCAL,1ROM-AGUA,01.__2021,"1,480.00",0,0.03
1,24,1ROMAYO,31,AGUASCAL,1ROM-AGUA,02.__2022,0,0,0
2,24,1ROMAYO,31,AGUASCAL,1ROM-AGUA,03.__2023,0,0,0
3,24,1ROMAYO,31,AGUASCAL,1ROM-AGUA,04.__2024,224,2.06,262.8
4,24,1ROMAYO,31,AGUASCAL,1ROM-AGUA,05.__2025,0,0,0
...,...,...,...,...,...,...,...,...,...
2284,46,VERACRUZ,45,POZARICA,VER-POZ,17.__2037,0,0,0
2285,46,VERACRUZ,45,POZARICA,VER-POZ,18.__2038,0,0,0
2286,46,VERACRUZ,45,POZARICA,VER-POZ,19.__2039,0,0,0
2287,46,VERACRUZ,45,POZARICA,VER-POZ,20.__2040,0,0,0


La descripción de cada variable es la siguiente:

- **Num_env** y **Nom_env** Son los identificadores, número y nombre, respectivamente; del nodo o región de envío (de acuerdo al orden mostrado en la Figura 4.3.2)
- **Num_env** y **Nom_env** Son los identificadores, número y nombre, respectivamente; del nodo o región de recepción (de acuerdo al orden mostrado en la Figura 4.3.2)
- **Enlace** Concatenación o identificación corta del enlace formado entre el nodo de envío y el nodo de recepción
- **Periodo** Identificación del año en el que el enlace se encontraría en operación. Cada enlace tiene 20 registros, uno por año, que van desde 2021 a 2041
- **Cap** Capacidad de flujo máximo que puede transmitir el enlace. El primer periodo corresponde a la capacidad actual y en los años subsecuentes se informa de incrementos o decrementos, si es que los hay
- **Real** Parte real de la impedancia eléctrica que tiene el enlace
- **Imag** Parte imaginaria de la impedancia eléctrica que tiene el enlace

El tipo de datos que se lee es el siguiente:

In [117]:
red.dtypes

Num_env     int64
Nom_env    object
Num_rec     int64
Nom_rec    object
Enlace     object
Periodo    object
Cap        object
Real       object
Img        object
dtype: object

Debido a que la variable de interés ("Cap") está en formato de objeto y se requiere en tipo entero o de punto flotante, se hace la limpieza de esta para quitar el formato que impide convertirla a numérica.

In [3]:
red['Cap'] = red['Cap'].str.replace(',','').astype(float)
red['Cap']

0       1480.0
1          0.0
2          0.0
3        224.0
4          0.0
         ...  
2284       0.0
2285       0.0
2286       0.0
2287       0.0
2288       0.0
Name: Cap, Length: 2289, dtype: float64

Esta base de datos se utiliza para hacer simulaciones en un programa de optimización más robusto, que evalua técnica y económicamente los Programas de Expansión de Generación y Transmisión (PEGyT), seleccionando de un abanico de alternativas (proyectos de infraestructura) las más eficientes en algún sentido: minimizar pérdidas de transmisión, reducción de emisiones de gases de efecto invernadero, maximización de ganancias en centrales eléctricas, entre otras. Así como respetando ciertas restricciones: cumplimiento de metas de generación renovable, política de confiabilidad (energía no suministrada y margen de reserva), operación dentro de los límites o capacidades de los enlaces, entre otras.

El resultado de este modelo es un plan de expansión de la transmisión y la generación que cumple con los planteamientos mencionados. Por ello, para cada año se puede tener un incremento o decremento de capacidad en los enlaces, que obedecerían a lo que el programa determinó en ese plan de expansión del sistema. 

Para nuestro ejercicio partiremos de la capacidad final que fue determinada por el PEGyT en el año horizonte (2041). Para ello habría que sumar a la capacidad actual (primer periodo) todas las adiciones y decrementos que se hayan presentado durante el de tiempo considerado en la planeación.

In [4]:
red['cve_enlace'] = red['Num_env'].map(str) + "-" + red['Num_rec'].map(str)
suma = red[['Cap','cve_enlace']].groupby(['cve_enlace']).aggregate(np.sum)
suma

,Cap
cve_enlace,
11-3,647.0
12-10,950.0
12-11,600.0
12-14,940.0
14-13,500.0
...,...
69-70,55.0
7-8,2575.0
8-9,2070.0


Se obtienen los arreglos de interés: Nodo de envío ("env"), Nodo de recepción ("rec") y Capacidad del enlace ("cap")

In [5]:
suma['Enlace'] = suma.index
env_rec = suma['Enlace'].str.split('-', expand=True)
env = env_rec[0].to_numpy()
rec = env_rec[1].to_numpy()
cap = suma['Cap'].to_numpy()

La dirección que tiene este grafo es la proporcionada en la base de datos, sin embargo, de acuerdo a nuestro objetivo y para resolver el problema de flujo máximo se partirá de una red dirigida, con las convenciones de sentido de flujo que se mencionaron líneas arriba. Dicha convención se puede ver de manera gráfica en la siguiente figura:

<p align = "center">
    <img src="images/red_dirigida.png" width="1329" height="911" />

        fuente: Elaborado por CENACE


Asignaremos las convenciones de las direcciones de las ramas que nos interesa para el ejercicio. En este caso, es en el sentido norte a sur, donde se observe una trayectoria de flujo desde el punto duente, en el noroeste del mapa (nodo 2), hasta el nodo sumidero, en el centro del país (nodo 40).

In [19]:
# Orden requerido de nodos de envío y recepción

env_conv = np.array([1,4,5,6,7,8,9,29,30,30,30,35,33,33,33,37,10,2,2,4,3,11,10,12,14,15,17,16,16,13,17,23,24,18,18,22,22,28,26,25,26,27,26,
31,32,36,34,38,38,38])
rec_conv = np.array([4,5,6,7,8,9,29,30,36,34,39,30,25,31,37,38,12,1,4,3,11,12,12,14,15,17,16,9,31,14,23,24,31,14,22,23,28,26,25,33,27,38,45,
32,34,34,38,40,44,42])


In [30]:
# Se generan las claves de los enlaces en la BD original "nodo de envío - nodo de recepción"
clave = []
for nodos in range(len(env)):
    x = env[nodos] + '-' + rec[nodos]
    clave.append(x)


In [44]:
# Se genera una clave de los nodos que no queremos que estén presentes en la lista de "claves" ya que implicarían el orden inverso 
# de envío y recepción 
errores = []
for nodos in range(len(env_conv)):
    x = str(rec_conv[nodos]) + '-' + str(env_conv[nodos])
    errores.append(x)

50

In [64]:
# Encontramos las posiciones de los enlaces "equivocados" o con orden invertido entre envío y recepción
c = [x for x in clave if x in errores]
pos = []
for elm in range(len(c)):
    x = clave.index(c[elm])
    pos.append(x)

[0, 1, 2, 4, 5, 6, 8, 16, 22, 23, 25, 33, 37, 47, 48, 54, 69, 86, 107]

In [65]:
# Se invierte el nodo de envío y nodo de recepción de los arreglos "env" y "rec"
for p in pos:
    e, r = env[p],rec[p]
    env[p] = r
    rec[p] = e

Generamos un DF que represente la matriz de incidencias

In [171]:
# Número de nodos en la red
env = env.astype("int")
rec = rec.astype("int")
nnodes = max(max(env), max(rec))
d = pd.DataFrame(0, index=np.arange(1,nnodes+1,1), columns=list(range(1,nnodes+1)))

In [172]:
aux = 0
for n in env:
    d.iloc[n-1,rec[aux]-1] = cap[aux]
    aux += 1

In [174]:
# Generamos el arreglo final de tipo "numpy array"
arreglo = d.to_numpy()
arreglo

array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [440.,   0.,   0., ...,   0.,   0.,   0.],
       [  0., 535.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,  55.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])